---
layout: post
title: Data storytelling, visualizando datos sobre reservas de hoteles.
date: 2025-05-11 10:49:19
author: Carlos Andrés Moreno
summary: En este artículo vamos a realizar un storytelling de un conjunto de datos de reservas de hoteles
categories: Data
thumbnail: plot
tags:
  - Data
  - Python
  - Plotly
---

# Introducción

Como parte de la Maestría en Cienicas de Datos que estoy estudiando en la [Universidad Oberta de Catalunya](https://uoc.edu), me fue asignada una actividad que consiste en crear un *Data Storytelling* de un conjunto de datos sobre reservaciones de hoteles en Lisboa, Portugal. Creo que es una excelente oportunidad de mostrar un poco lo que he aprendido este semestre en la asignatura de Visualización de Datos, es por eso que he decidido crear el *storytelling* en un nuevo artículo del blog.


# Sobre los datos

El conjunto de datos que vamos a usar a lo largo del *storytelling* ha sido descrito completamente en el artículo titulado [Hotel booking demand datasets ](https://www.sciencedirect.com/science/article/pii/S2352340918315191). Como se ha mencionado anteriormente, los datos tratan sobre reservaciones de hoteles en Lisboa, Portugal hechas desde Julio 1 de 2015 hasta Agosto 31 de 2017. Particularmente dos hoteles, uno ubicado en la zona urbana de la ciudad y otro hotel tipo resort.

Cada fila del conjunto de datos es una reservación hecha por un cliente para alguno de los dos hoteles. Además, tenemos información sobre el número de niños, bebés y adultos dentro de la reserva; més, semana y día en el cual se hizo la reserva; país de residencia de cliente, si la reserva se canceló o no, el tipo de comida que se incluyó para los clientes, entre otra información de interés.

# Herrmientas usadas para la visualización

Todas las visualizaciones que vas a ver han sido creadas usando la librería [Plotly](https://plotly.com/python/). El blog está hosteado en Github pages y utiliza Jekyll para generar todo el sitio (si quieres saber un poco más a cerca de esto, puedes hacer click [acá](https://carmoreno.com.co/tutoriales/2015/08/13/Como-hice-el-blog/)).

Me gustaría tambien mencionar a [Jérome Eertmans](https://github.com/jeertmans), el artículo de su blog titulado ["Embedding Plotly in a Jekyll Post"](https://eertmans.be/posts/plotly-example/) fue de gran ayuda para poder integrar Plotly y Jekyll sin morir en el intento (a veces una buena googleada es mejor que escribirle a Gemini o ChatGPT).

# Ahora sí, manos a la obra.

### ¿Prefieres mucha o poca gente?

Lisboa es uno de los destinos turísticos más visitados todos los años en Europa, tiene un clima excelente, buena comida, hermosos paisajes y playas mediterraneas. En 2021, esta ciudad ocupó [el tercer lugar](https://elordenmundial.com/mapas-y-graficos/ciudades-mas-visitadas-europa/) entre las 15 ciudades más visitadas de Europa, solo por detrás de París y Barcelona. Que sea tan popular no significa que todo sea color de rosa; muchas veces implica sitios de interés repletos de gente, pocos lugares para parquear tu vehículo y pagar precios más altos por temporada. ¿No sería mejor ir a Lisboa en una temporada un poco mas relajada?, podriamos usar los datos de hoteles para saber cuando hay mayor cantidad de turistas, entre más reservaciones sin cancelación existan, significa mayor cantidad de personas yendo a estos hoteles. Miremos qué nos dicen los datos.

In [21]:
import polars as pl
import numpy as np
from plotting import express as px
from plotting import graph_objects as go
from plotly.subplots import make_subplots

In [80]:
reservas = pl.read_csv("/Users/carlosm/Documents/MDS/4th Semester/Viusalizacion/PEC3/hotel_bookings.csv", null_values=["NA"])
reservas = reservas.with_columns(
    pl.concat_str(
        pl.col("arrival_date_year"),
        pl.col("arrival_date_month"),
        pl.lit(1),
        separator="-"
    ).str.strptime(pl.Date, "%Y-%B-%d").alias("arrival_date")
)

In [81]:
reservas_por_mes = (
    reservas
        .filter(pl.col("is_canceled") == 0)
        .select("arrival_date", "hotel", "adr", "arrival_date_month", "country", "adults", "children", "is_canceled")
)
reservas_por_mes

arrival_date,hotel,adr,arrival_date_month,country,adults,children,is_canceled
date,str,f64,str,str,i64,i64,i64
2015-07-01,"""Resort Hotel""",0.0,"""July""","""PRT""",2,0,0
2015-07-01,"""Resort Hotel""",0.0,"""July""","""PRT""",2,0,0
2015-07-01,"""Resort Hotel""",75.0,"""July""","""GBR""",1,0,0
2015-07-01,"""Resort Hotel""",75.0,"""July""","""GBR""",1,0,0
2015-07-01,"""Resort Hotel""",98.0,"""July""","""GBR""",2,0,0
…,…,…,…,…,…,…,…
2017-08-01,"""City Hotel""",96.14,"""August""","""BEL""",2,0,0
2017-08-01,"""City Hotel""",225.43,"""August""","""FRA""",3,0,0
2017-08-01,"""City Hotel""",157.71,"""August""","""DEU""",2,0,0


In [5]:
numero_reservas_por_mes = (
    reservas_por_mes
    .group_by("arrival_date_month", "month", "hotel")
        .agg(pl.col("adr").count())
        .rename({"adr": "count"})
        .sort(by="month")
)

numero_reservas_por_mes


arrival_date_month,month,hotel,count
str,i8,str,u32
"""January""",1,"""Resort Hotel""",1868
"""January""",1,"""City Hotel""",2254
"""February""",2,"""City Hotel""",3064
"""February""",2,"""Resort Hotel""",2308
"""March""",3,"""Resort Hotel""",2573
…,…,…,…
"""October""",10,"""Resort Hotel""",2577
"""November""",11,"""City Hotel""",2696
"""November""",11,"""Resort Hotel""",1976


In [6]:
fig = px.bar(numero_reservas_por_mes, x="arrival_date_month", y="count", color="hotel", title="Cantidad de reservas sin cancelación por mes (2015 - 2017)", opacity=0.6, labels={"hotel": "Hotel", "arrival_date_month": "Mes de llegada", "count": "Cantidad de reservas"})
fig.show()

Al parecer, los meses donde más reservaciones hay son Julio y Agosto, tanto para el Hotel Urbano como para el Hotel Resort. No es nada nuevo, puesto que son normalmente los meses de verano donde más personas salen a vacacionar. Sin embargo, algo que sorprende son los números de reservas para el mes de Junio, ya que son pocas en comparación con el increible clima que hace en este mes. Los primeros días de Septiembre, si bien no acostumbra a ser extremadamente soleado, no esta nada mal para tomarse unas vacaciones. Por lo general, los primeros cuatro meses del año y los últimos tres, no tienen el mejor clima, por eso no es de sorprendernos que sus números no sean tan altos. Por otro lado, sería interesante ver las nacionalidades de los turistas que vistan estos hoteles, miremos:

In [7]:
nacionalidades_reservas = (
    reservas_por_mes.select("country", "hotel", "adr")
        .group_by(["country", "hotel"])
        .agg(pl.col("adr").count())
        .rename({"adr": "count"})
        .sort(by="count", descending=True)
)
nacionalidades_reservas

country,hotel,count
str,str,u32
"""PRT""","""City Hotel""",10879
"""PRT""","""Resort Hotel""",10192
"""FRA""","""City Hotel""",7081
"""GBR""","""Resort Hotel""",5923
"""DEU""","""City Hotel""",5012
…,…,…
"""ZMB""","""Resort Hotel""",1
"""CYM""","""Resort Hotel""",1
"""EGY""","""Resort Hotel""",1


In [8]:
fig = px.bar(nacionalidades_reservas, x="country", y="count", color="hotel", title="Cantidad de reservaciones no cancelandas por país de origen (2015 - 2017)", opacity=0.6, labels={"hotel": "Hotel", "country": "País", "count": "Cantidad de reservas"})
fig.show()

Vemos que tenemos muchos paises (178 para ser exactos) desde donde proceden las personas que han hecho las reservas. Tiene sentido que Portugal sea el país desde donde más se hacen reservas a Lisboa, seguido de los paises cercanos como España, Francia, Italia y Reino Unido, este último al igual que Irlanda parecen preferir el hotel de tipo resort en lugar del hotel urbano. Por el contrario, los tres primeros paises mencionados anteriormente prefieren visitar el Hotel Urbano, al igual que Alemania. Sería interesante conocer de que paises serían los turistas que nos pudieramos encontrar en Lisboa, en caso de que viajaramos en Junio o Septiembre (que como ya vimos, son los mejores meses para viajar), en caso de que nos hospedemos en alguno de los dos hoteles. Quizá podamos hacer amigos mientras viajamos, miremos qué nos dicen los datos:

In [9]:
nacionalidad_reservas_mes = (
    reservas_por_mes
    .filter(pl.col("arrival_date_month").is_in(["June", "September"]))
    .group_by("arrival_date_month", "country", "hotel")
    .agg(pl.col("adr").count(), pl.col("adults").sum(), pl.col("children").sum())
    .rename({"adr": "count"})
    .filter(pl.col("count") > 50)
    .sort(by=["arrival_date_month", "count"], descending=True)
)
nacionalidad_reservas_mes = nacionalidad_reservas_mes.with_columns(
    total_people=(pl.col("adults") + pl.col("children"))
)
nacionalidad_reservas_mes
#reservas_por_mes

arrival_date_month,country,hotel,count,adults,children,total_people
str,str,str,u32,i64,i64,i64
"""September""","""PRT""","""City Hotel""",1196,2005,57,2062
"""September""","""GBR""","""Resort Hotel""",568,1089,26,1115
"""September""","""FRA""","""City Hotel""",519,943,11,954
"""September""","""DEU""","""City Hotel""",447,826,17,843
"""September""","""PRT""","""Resort Hotel""",430,797,40,837
…,…,…,…,…,…,…
"""June""","""SWE""","""City Hotel""",68,128,15,143
"""June""","""SRB""","""City Hotel""",68,74,0,74
"""June""","""DEU""","""Resort Hotel""",56,107,2,109


In [10]:
fig = px.treemap(
    nacionalidad_reservas_mes,
    path=[px.Constant("Months"), 'arrival_date_month', "hotel", 'country'],
    values='total_people',
    color='total_people',
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=np.average(
        (nacionalidad_reservas_mes.select('total_people').get_column("total_people")).to_list()
    ),
    title="Cantidad de personas por país y tipo de hotel en los meses de Junio y Septiembre (2015 - 2017), incluyendo adultos y niños (no bebés).",
    labels={"total_people": "Total Personas"},
)
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.update_traces(
    marker=dict(cornerradius=5),
    hovertemplate='<b>%{label} </b> <br> Total personas: %{value:,}'
)
fig.show()

El anterior gráfico de árbol nos muestra la cantidad de personas por nacionalidad que podemos encontrarnos, sea que vayamos en Junio o en Septiembre y sea que nos quedemos en el hotel tipo resort o en el hotel urbano. Puedes pasar el cursor por encima de cada cuadro para obtener el total de personas de cada nacionalidad, también hacer clic por ejemplo en el hotel urbano del mes de Junio, para navegar directamente a esta información.

Vemos que si nos hospedamos en el hotel de tipo resort en cualquiera de los dos meses, tenemos muchas más chances de encontrarnos Británicos que Portugueses. Por el contrario, en el hotel urbano nos encontraremos en mayor medida con Portugueses, Alemanes y Frances. En Septiembre podriamos encontrarnos con algún Español en el hotel urbano. Parece ser que los Británicos tienden a preferir el hotel de tipo resort.

Del anterior gráfico tambien podemos observar que habrá un poco más de turistas en Junio que en Septiembre, sin embargo, será el hotel urbano quién tendrá mas cantidad de personas sin importar el mes. Por lo tanto, si tu plan es relajarte en un hotel cercano a la playa y no tanto salir a conocer la ciudad llena de turistas, sería un excelente plan ir al resort cerca del mar en Junio ya que habrán menos personas, y si hablas Inglés, podrías aprovechar que la mayoría de ellos serán Británicos.

### ¡Atentos a las cancelaciones!

Ya vimos que Junio y Septiembre pueden ser los mejores meses para viajar si es que estás buscando un equilibrio entre clima y cantidad de turistas, pero quizá no te importe mucho esto último. Quizá lo más importante para tí sea viajar en los meses de verano, pero no siempre es posible reservar a tiempo y los mejores meses son los primeros en irse. Que tal si miramos la taza de cancelaciones por mes para mirar si, de no poder reservar a tiempo, tenemos algún chance de coger alguna reserva cancelada e irnos de vacaciones a Lisboa.

In [86]:
reservas_canceladas = (
    reservas
        .select(
            "arrival_date_month",
            "hotel",
            "country",
            pl.col("arrival_date").dt.month().alias("month"),
            pl.when(pl.col.is_canceled == 1).then(pl.lit("Si")).otherwise(pl.lit("No")).alias("is_canceled"),
        )
)
reservas_canceladas
#total = reservas_canceladas.select("is_canceled").count()

arrival_date_month,hotel,country,month,is_canceled
str,str,str,i8,str
"""July""","""Resort Hotel""","""PRT""",7,"""No"""
"""July""","""Resort Hotel""","""PRT""",7,"""No"""
"""July""","""Resort Hotel""","""GBR""",7,"""No"""
"""July""","""Resort Hotel""","""GBR""",7,"""No"""
"""July""","""Resort Hotel""","""GBR""",7,"""No"""
…,…,…,…,…
"""August""","""City Hotel""","""BEL""",8,"""No"""
"""August""","""City Hotel""","""FRA""",8,"""No"""
"""August""","""City Hotel""","""DEU""",8,"""No"""


In [87]:
reservas_canceladas_por_mes = (
    reservas_canceladas
        .group_by("arrival_date_month", "month", "hotel", "is_canceled")
        .agg(pl.col("country").count())
        .rename({"country": "count"})
        .sort(by="month")
)
reservas_canceladas_por_mes

arrival_date_month,month,hotel,is_canceled,count
str,i8,str,str,u32
"""January""",1,"""City Hotel""","""Si""",1482
"""January""",1,"""Resort Hotel""","""Si""",325
"""January""",1,"""Resort Hotel""","""No""",1868
"""January""",1,"""City Hotel""","""No""",2254
"""February""",2,"""Resort Hotel""","""Si""",795
…,…,…,…,…
"""November""",11,"""City Hotel""","""Si""",1661
"""December""",12,"""Resort Hotel""","""Si""",631
"""December""",12,"""City Hotel""","""Si""",1740


In [110]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "sunburst"}, {"type": "sunburst"}]],
    subplot_titles=("Cancelaciones por mes: Hotel Resort", "Cancelaciones por mes: Hotel Urbano")
)
for hotel_type, coord in [("Resort Hotel", {"row": 1, "col": 1}), ("City Hotel", {"row": 1, "col": 2})]:
    subplot = px.sunburst(
        reservas_canceladas_por_mes.filter(pl.col("hotel") == hotel_type).sort(by="month"),
        path=['hotel', 'arrival_date_month', "is_canceled"], values='count', color="arrival_date_month",
        color_discrete_sequence=["white"]+px.colors.qualitative.Light24)
    fig.add_trace(subplot.data[0], **coord)
fig.update_layout(title_text="Total de cancelaciones por mes (2015 - 2017)")
fig.show()

In [20]:
x = ['1970-01-01', '1970-01-01', '1970-02-01', '1970-04-01', '1970-01-02',
     '1972-01-31', '1970-02-13', '1971-04-19']
fig = make_subplots(rows=3, cols=2)

trace0 = go.Histogram(x=x, nbinsx=4)
trace1 = go.Histogram(x=x, nbinsx = 8)
trace2 = go.Histogram(x=x, nbinsx=10)
trace3 = go.Histogram(x=x,
                      xbins=dict(
                      start='1969-11-15',
                      end='1972-03-31',
                      size='M18'), # M18 stands for 18 months
                      autobinx=False
                     )
trace4 = go.Histogram(x=x,
                      xbins=dict(
                      start='1969-11-15',
                      end='1972-03-31',
                      size='M4'), # 4 months bin size
                      autobinx=False
                      )
trace5 = go.Histogram(x=x,
                      xbins=dict(
                      start='1969-11-15',
                      end='1972-03-31',
                      size= 'M2'), # 2 months
                      autobinx = False
                      )

fig.add_trace(trace0, 1, 1)
fig.add_trace(trace1, 1, 2)
fig.add_trace(trace2, 2, 1)
fig.add_trace(trace3, 2, 2)
fig.add_trace(trace4, 3, 1)
fig.add_trace(trace5, 3, 2)

fig.show()